# Análise de Artigos Científicos com Gemini AI

## Escopo: Nanorevestimentos e Tintas (Nanocoatings and Paints)

Este notebook realiza análise automatizada de artigos científicos para identificar trabalhos relevantes ao escopo de nanorevestimentos e tintas usando a API do Google Gemini.

### Funcionalidades:
1. **Instalação de dependências**
2. **Configuração da API Gemini**
3. **Definição de palavras-chave específicas**
4. **Busca e filtro de artigos**
5. **Análise individual de abstracts**
6. **Processamento em lote com rate limiting**
7. **Sistema de checkpoint para recuperação**




In [22]:
# Instalação das dependências necessárias
!pip install google-generativeai pandas

In [23]:
# Imports necessários
import pandas as pd
import google.generativeai as genai
import time
import random
import os
from typing import List, Optional

# Função para carregar API key do arquivo
def carregar_api_key(arquivo_api='api_key.txt'):
    """
    Carrega a API key do arquivo especificado
    
    Args:
        arquivo_api: Caminho para o arquivo contendo a API key
    
    Returns:
        String com a API key ou None se houver erro
    """
    try:
        # Verificar se o arquivo existe
        if not os.path.exists(arquivo_api):
            print(f"❌ Arquivo '{arquivo_api}' não encontrado!")
            print(f"💡 Crie o arquivo '{arquivo_api}' e coloque sua API key nele")
            return None
        
        # Ler a API key do arquivo
        with open(arquivo_api, 'r', encoding='utf-8') as file:
            api_key = file.read().strip()
        
        # Verificar se a API key não está vazia
        if not api_key:
            print(f"❌ Arquivo '{arquivo_api}' está vazio!")
            return None
        
        # Verificar formato básico da API key (deve começar com 'AIza')
        if not api_key.startswith('AIza'):
            print(f"⚠️ API key pode estar no formato incorreto")
            print(f"   APIs do Google Gemini geralmente começam com 'AIza'")
        
        print(f"✅ API key carregada com sucesso de '{arquivo_api}'")
        print(f"🔑 API key: {api_key[:10]}...{api_key[-4:]} (ocultada por segurança)")
        
        return api_key
        
    except FileNotFoundError:
        print(f"❌ Arquivo '{arquivo_api}' não encontrado!")
        print(f"💡 Crie o arquivo '{arquivo_api}' e coloque sua API key nele")
        return None
    except PermissionError:
        print(f"❌ Sem permissão para ler o arquivo '{arquivo_api}'")
        return None
    except Exception as e:
        print(f"❌ Erro ao carregar API key: {e}")
        return None

# Carregar e configurar a API do Gemini
print("🔐 Carregando configurações da API...")

api_key = carregar_api_key('api_key.txt')

if api_key:
    # Configurar a API do Gemini
    genai.configure(api_key=api_key)
    print("✅ API do Gemini configurada com sucesso")
    
    # Testar a conexão (opcional)
    try:
        # Fazer um teste simples para verificar se a API está funcionando
        model = genai.GenerativeModel('gemini-2.0-flash-exp')
        test_response = model.generate_content("Hello")
        print("🔗 Conexão com a API testada com sucesso")
    except Exception as e:
        print(f"⚠️ Erro ao testar a API: {e}")
        print("   A API key pode estar inválida ou sem créditos")
        
else:
    print("❌ Não foi possível configurar a API")
    print("\n📋 Para configurar a API:")
    print("1. Crie um arquivo chamado 'api_key.txt' no mesmo diretório deste notebook")
    print("2. Cole sua API key do Google Gemini no arquivo (apenas a key, sem aspas)")
    print("3. Salve o arquivo e execute novamente esta célula")
    print("\n🔗 Como obter uma API key:")
    print("   Acesse: https://makersuite.google.com/app/apikey")

print("\n✅ Dependências carregadas")

🔐 Carregando configurações da API...
❌ Arquivo 'api_key.txt' não encontrado!
💡 Crie o arquivo 'api_key.txt' e coloque sua API key nele
❌ Não foi possível configurar a API

📋 Para configurar a API:
1. Crie um arquivo chamado 'api_key.txt' no mesmo diretório deste notebook
2. Cole sua API key do Google Gemini no arquivo (apenas a key, sem aspas)
3. Salve o arquivo e execute novamente esta célula

🔗 Como obter uma API key:
   Acesse: https://makersuite.google.com/app/apikey

✅ Dependências carregadas


In [24]:
# Criar arquivo .gitignore para proteger a API key
def criar_gitignore():
    """
    Cria ou atualiza arquivo .gitignore para proteger arquivos sensíveis
    """
    gitignore_content = """# API Keys e arquivos sensíveis
api_key.txt
*.key
.env

# Arquivos de checkpoint
*checkpoint*.csv

# Arquivos temporários
*.tmp
*.temp

# Resultados de análises (opcional - remova se quiser versionar)
*_resultado.csv
*_analise.csv
nanomateriais_identificados*.csv

# Jupyter Notebook checkpoints
.ipynb_checkpoints/

# Python cache
__pycache__/
*.pyc
*.pyo
"""
    
    try:
        with open('.gitignore', 'w', encoding='utf-8') as file:
            file.write(gitignore_content)
        print("✅ Arquivo .gitignore criado/atualizado com sucesso")
        print("🔒 Seus arquivos sensíveis estão protegidos do versionamento")
    except Exception as e:
        print(f"⚠️ Erro ao criar .gitignore: {e}")

# Executar a criação do .gitignore
criar_gitignore()

# Função auxiliar para criar arquivo de exemplo da API key
def criar_exemplo_api_key():
    """
    Cria um arquivo de exemplo mostrando como configurar a API key
    """
    exemplo_content = """AIzaSyA_exemplo_da_sua_api_key_aqui_substitua_por_sua_key_real

INSTRUÇÕES:
1. Substitua o texto acima pela sua API key real do Google Gemini
2. Mantenha apenas a API key (sem aspas, sem espaços extras)
3. Salve este arquivo como 'api_key.txt'
4. Delete esta linha de instruções

Como obter sua API key:
- Acesse: https://makersuite.google.com/app/apikey
- Faça login com sua conta Google
- Clique em "Create API Key"
- Copie a key gerada e cole aqui
"""
    
    try:
        if not os.path.exists('api_key.txt'):
            with open('api_key_exemplo.txt', 'w', encoding='utf-8') as file:
                file.write(exemplo_content)
            print("📝 Arquivo 'api_key_exemplo.txt' criado")
            print("   Renomeie para 'api_key.txt' e adicione sua key real")
    except Exception as e:
        print(f"⚠️ Erro ao criar exemplo: {e}")

# Criar exemplo se não existir api_key.txt
if not os.path.exists('api_key.txt'):
    print("\n📝 API key não encontrada. Criando arquivo de exemplo...")
    criar_exemplo_api_key()

✅ Arquivo .gitignore criado/atualizado com sucesso
🔒 Seus arquivos sensíveis estão protegidos do versionamento

📝 API key não encontrada. Criando arquivo de exemplo...
📝 Arquivo 'api_key_exemplo.txt' criado
   Renomeie para 'api_key.txt' e adicione sua key real


## 1. Configuração de Parâmetros

Definição dos parâmetros principais para análise e rate limiting da API.

In [25]:
# Configurações principais
CONFIG = {
    'modelo_gemini': 'gemini-2.0-flash-exp',  # Flash-Lite para rate limits melhores
    'max_requests_per_minute': 20,              # Limite de requisições por minuto
    'max_requests_per_day': 1000,               # Limite diário recomendado
    'delay_between_requests': 5,              # Delay entre requisições (segundos)
    'arquivo_entrada': '/home/delon/Modelos/cenanoink/Projeto-CENanoInk/Alan Delon.csv',
    'checkpoint_file': 'analise_escopo_checkpoint.csv',
    'arquivo_saida': 'df_com_analise_escopo_completo.csv'
}

print("📋 Configurações carregadas:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

📋 Configurações carregadas:
  modelo_gemini: gemini-2.0-flash-exp
  max_requests_per_minute: 20
  max_requests_per_day: 1000
  delay_between_requests: 5
  arquivo_entrada: /home/delon/Modelos/cenanoink/Projeto-CENanoInk/Alan Delon.csv
  checkpoint_file: analise_escopo_checkpoint.csv
  arquivo_saida: df_com_analise_escopo_completo.csv


In [ ]:

# VERSÃO OTIMIZADA - Identificação de Nanomateriais
import pandas as pd
import google.generativeai as genai
import time
import re
from collections import Counter
from typing import Set, Dict, List

# Configuração otimizada
CONFIG_OTIMIZADO = {
    'modelo_gemini': 'gemini-2.0-flash-exp',
    'max_requests_per_minute': 25,
    'delay_between_requests': 2.5,
    'batch_size': 50,  # Processar em lotes maiores
    'verbose': False   # Reduzir verbosidade
}

# Nanomateriais organizados e otimizados
NANOMATERIAIS_OTIMIZADO = {
    'oxidos_metalicos': [
        'TiO2', 'titanium dioxide', 'ZnO', 'zinc oxide',
        'Al2O3', 'alumina', 'SiO2', 'silica', 'Fe2O3', 'iron oxide',
        'CeO2', 'cerium oxide', 'SnO2', 'tin oxide'
    ],
    'nanoparticulas_metalicas': [
        'silver nanoparticle', 'AgNP', 'gold nanoparticle', 'AuNP',
        'copper nanoparticle', 'CuNP', 'iron nanoparticle', 'FeNP'
    ],
    'nanomateriais_carbono': [
        'carbon nanotube', 'CNT', 'graphene', 'graphene oxide',
        'fullerene', 'carbon black', 'carbon nanofiber'
    ],
    'outros': [
        'quantum dot', 'nanoclay', 'nanocellulose', 'polymer nanocomposite'
    ]
}

def criar_patterns_regex() -> Dict[str, re.Pattern]:
    """Cria patterns regex otimizados para busca rápida"""
    patterns = {}
    for categoria, materiais in NANOMATERIAIS_OTIMIZADO.items():
        # Criar pattern regex case-insensitive
        pattern_str = '|'.join(re.escape(mat) for mat in materiais)
        patterns[categoria] = re.compile(f'({pattern_str})', re.IGNORECASE)
    return patterns

def identificar_nanomateriais_rapido(abstract_text: str, patterns: Dict[str, re.Pattern]) -> Set[str]:
    """Identificação rápida usando regex - sem Gemini"""
    if pd.isna(abstract_text) or not str(abstract_text).strip():
        return set()
    
    materiais_encontrados = set()
    text = str(abstract_text)
    
    for categoria, pattern in patterns.items():
        matches = pattern.findall(text)
        materiais_encontrados.update(matches)
    
    return materiais_encontrados

def identificar_nanomateriais_gemini_batch(abstracts: List[str]) -> List[str]:
    """Processamento em lote com Gemini para maior eficiência"""
    if not abstracts:
        return []
    
    # Combinar múltiplos abstracts em uma única requisição
    combined_text = "\n\n---ABSTRACT---\n\n".join(abstracts[:5])  # Máximo 5 por requisição
    
    model = genai.GenerativeModel(CONFIG_OTIMIZADO['modelo_gemini'])
    
    prompt = f"""
    Analise os abstracts científicos separados por "---ABSTRACT---" e identifique nanomateriais em cada um.
    
    {combined_text}
    
    Para cada abstract, liste os nanomateriais encontrados ou "NENHUM" se não houver.
    Formato de resposta:
    ABSTRACT 1: material1, material2
    ABSTRACT 2: NENHUM
    ABSTRACT 3: material3
    
    Use nomenclatura padrão (TiO2, ZnO, graphene, etc.)
    """
    
    try:
        response = model.generate_content(prompt)
        return processar_resposta_batch(response.text, len(abstracts[:5]))
    except Exception as e:
        print(f"Erro Gemini batch: {e}")
        return ["Erro"] * len(abstracts[:5])

def processar_resposta_batch(response_text: str, num_abstracts: int) -> List[str]:
    """Processa resposta em lote do Gemini"""
    linhas = response_text.strip().split('\n')
    resultados = []
    
    for i in range(num_abstracts):
        encontrado = False
        for linha in linhas:
            if f"ABSTRACT {i+1}:" in linha:
                material_part = linha.split(':', 1)[1].strip()
                if material_part.upper() == "NENHUM":
                    resultados.append("Nenhum nanomaterial identificado")
                else:
                    resultados.append(material_part)
                encontrado = True
                break
        
        if not encontrado:
            resultados.append("Nenhum nanomaterial identificado")
    
    return resultados

def processar_dataframe_otimizado(df: pd.DataFrame, 
                                use_gemini: bool = False,
                                max_linhas: int = None) -> pd.DataFrame:
    """Versão otimizada do processamento"""
    
    if 'Abstract' not in df.columns:
        print("❌ Coluna 'Abstract' não encontrada")
        return df
    
    # Limitar processamento se especificado
    if max_linhas:
        df_work = df.head(max_linhas).copy()
    else:
        df_work = df.copy()
    
    print(f"🚀 Processando {len(df_work)} abstracts (modo otimizado)")
    
    # Criar patterns regex uma vez
    patterns = criar_patterns_regex()
    
    # Inicializar coluna
    df_work['Nanomaterial Citado'] = ''
    
    # FASE 1: Busca rápida com regex (todos os abstracts)
    print("📍 Fase 1: Busca rápida por palavras-chave...")
    start_time = time.time()
    
    for idx in range(len(df_work)):
        abstract = df_work.iloc[idx]['Abstract']
        materiais = identificar_nanomateriais_rapido(abstract, patterns)
        
        if materiais:
            resultado = ', '.join(sorted(materiais))
        else:
            resultado = "Nenhum nanomaterial identificado"
        
        df_work.iloc[idx, df_work.columns.get_loc('Nanomaterial Citado')] = resultado
        
        # Progresso a cada 100 linhas
        if (idx + 1) % 100 == 0:
            print(f"  Processado: {idx + 1}/{len(df_work)}")
    
    fase1_time = time.time() - start_time
    print(f"✅ Fase 1 concluída em {fase1_time:.1f}s")
    
    # FASE 2: Gemini apenas para abstracts sem nanomateriais (opcional)
    if use_gemini:
        abstracts_sem_materiais = df_work[
            df_work['Nanomaterial Citado'] == "Nenhum nanomaterial identificado"
        ]
        
        if len(abstracts_sem_materiais) > 0:
            print(f"📍 Fase 2: Análise Gemini para {len(abstracts_sem_materiais)} abstracts sem materiais...")
            
            abstracts_list = abstracts_sem_materiais['Abstract'].tolist()
            indices_list = abstracts_sem_materiais.index.tolist()
            
            # Processar em lotes
            for i in range(0, len(abstracts_list), 5):
                batch_abstracts = abstracts_list[i:i+5]
                batch_indices = indices_list[i:i+5]
                
                resultados = identificar_nanomateriais_gemini_batch(batch_abstracts)
                
                # Atualizar DataFrame
                for idx, resultado in zip(batch_indices, resultados):
                    df_work.iloc[
                        df_work.index.get_loc(idx), 
                        df_work.columns.get_loc('Nanomaterial Citado')
                    ] = resultado
                
                # Rate limiting
                time.sleep(CONFIG_OTIMIZADO['delay_between_requests'])
                
                if (i + 5) % 25 == 0:
                    print(f"  Gemini: {min(i + 5, len(abstracts_list))}/{len(abstracts_list)}")
    
    return df_work

def gerar_estatisticas_rapidas(df: pd.DataFrame) -> None:
    """Gera estatísticas de forma otimizada"""
    if 'Nanomaterial Citado' not in df.columns:
        print("❌ Coluna não encontrada")
        return
    
    # Contar materiais
    todos_materiais = []
    for materiais_str in df['Nanomaterial Citado'].dropna():
        if materiais_str and "nenhum" not in materiais_str.lower() and "erro" not in materiais_str.lower():
            materiais = [m.strip() for m in materiais_str.split(',')]
            todos_materiais.extend(materiais)
    
    contador = Counter(todos_materiais)
    
    print(f"\n📊 ESTATÍSTICAS RÁPIDAS")
    print(f"Total de artigos: {len(df):,}")
    print(f"Artigos com nanomateriais: {len(df[df['Nanomaterial Citado'] != 'Nenhum nanomaterial identificado']):,}")
    print(f"Materiais únicos identificados: {len(contador)}")
    
    if contador:
        print(f"\n🏆 Top 5 nanomateriais:")
        for i, (material, count) in enumerate(contador.most_common(5), 1):
            print(f"  {i}. {material}: {count} menções")

# EXECUÇÃO OTIMIZADA
print("🚀 VERSÃO OTIMIZADA - IDENTIFICAÇÃO DE NANOMATERIAIS")
print("="*60)

# Carregar dados
try:
    df = pd.read_csv(CONFIG['arquivo_entrada'])
    print(f"✅ {len(df):,} artigos carregados")
    
    if 'Abstract' in df.columns:
        abstracts_validos = df['Abstract'].notna().sum()
        print(f"📄 {abstracts_validos:,} abstracts disponíveis")
        
        # Opções de processamento
        print("\nOpções:")
        print("1. Apenas busca rápida (regex) - Recomendado")
        print("2. Busca rápida + Gemini para casos não identificados")
        print("3. Teste com 50 abstracts")
        
        opcao = input("Escolha (1/2/3): ")
        
        if opcao == "3":
            # Teste rápido
            print("\n🧪 TESTE RÁPIDO (50 abstracts)")
            df_result = processar_dataframe_otimizado(df, use_gemini=False, max_linhas=50)
            gerar_estatisticas_rapidas(df_result)
            
        elif opcao == "1":
            # Processamento rápido completo
            print("\n🚀 PROCESSAMENTO COMPLETO (só regex)")
            df_result = processar_dataframe_otimizado(df, use_gemini=False)
            
            # Salvar
            arquivo_saida = 'nanomateriais_identificados_rapido.csv'
            df_result.to_csv(arquivo_saida, index=False)
            print(f"💾 Salvo em: {arquivo_saida}")
            
            gerar_estatisticas_rapidas(df_result)
            
        elif opcao == "2":
            # Processamento híbrido
            print("\n🤖 PROCESSAMENTO HÍBRIDO (regex + Gemini)")
            df_result = processar_dataframe_otimizado(df, use_gemini=True)
            
            # Salvar
            arquivo_saida = 'nanomateriais_identificados_hibrido.csv'
            df_result.to_csv(arquivo_saida, index=False)
            print(f"💾 Salvo em: {arquivo_saida}")
            
            gerar_estatisticas_rapidas(df_result)
    
    else:
        print("❌ Coluna 'Abstract' não encontrada")
        
except Exception as e:
    print(f"❌ Erro: {e}")

🚀 VERSÃO OTIMIZADA - IDENTIFICAÇÃO DE NANOMATERIAIS
✅ 525 artigos carregados
📄 525 abstracts disponíveis

Opções:
1. Apenas busca rápida (regex) - Recomendado
2. Busca rápida + Gemini para casos não identificados
3. Teste com 50 abstracts

🚀 PROCESSAMENTO COMPLETO (só regex)
🚀 Processando 525 abstracts (modo otimizado)
📍 Fase 1: Busca rápida por palavras-chave...
  Processado: 100/525
  Processado: 200/525
  Processado: 300/525
  Processado: 400/525
  Processado: 500/525
✅ Fase 1 concluída em 0.7s
💾 Salvo em: nanomateriais_identificados_rapido.csv

📊 ESTATÍSTICAS RÁPIDAS
Total de artigos: 525
Artigos com nanomateriais: 179
Materiais únicos identificados: 39

🏆 Top 5 nanomateriais:
  1. TiO2: 42 menções
  2. ZnO: 25 menções
  3. Al2O3: 24 menções
  4. carbon nanotube: 20 menções
  5. alumina: 19 menções


## 2. Definição de Palavras-Chave

Conjunto abrangente de termos relacionados ao escopo de nanorevestimentos e tintas.

In [ ]:
# Palavras-chave organizadas por categoria
PALAVRAS_CHAVE = {
    'termos_principais': [
        "nanocoating", "nanocoatings", "nano coating", "nano coatings",
        "nanorevestimento", "nanorevestimentos", "nanocomposite coating"
    ],
    
    'materiais_nano': [
        "TiO2", "titanium dioxide", "zinc oxide", "ZnO", 
        "silica nanoparticle", "alumina nanoparticle", "Al2O3",
        "carbon nanotube", "CNT", "graphene", "iron oxide", "Fe2O3",
        "cerium oxide", "CeO2", "clay nanoparticles"
    ],
    
    'propriedades_funcionais': [
        "anticorrosive coating", "anti-corrosion coating",
        "antimicrobial coating", "antibacterial coating",
        "self-cleaning coating", "superhydrophobic coating",
        "photocatalytic coating", "UV resistant coating",
        "scratch resistant coating", "wear resistant coating"
    ],
    
    'tecnicas_preparacao': [
        "sol-gel coating", "layer-by-layer", "dip coating",
        "spin coating", "spray coating", "electrodeposition",
        "CVD coating", "PVD coating", "plasma treatment"
    ],
    
    'aplicacoes': [
        "protective coating", "functional coating",
        "smart coating", "intelligent coating", "automotive coating",
        "marine coating", "architectural coating", "biomedical coating"
    ]
}

# Criar lista única de todas as palavras-chave
todas_palavras_chave = []
for categoria, palavras in PALAVRAS_CHAVE.items():
    todas_palavras_chave.extend(palavras)

print(f"🔍 Total de palavras-chave definidas: {len(todas_palavras_chave)}")
print(f"📂 Categorias: {list(PALAVRAS_CHAVE.keys())}")

## 3. Funções Utilitárias

Funções para filtro, busca e manipulação de dados.

In [ ]:
# ...existing code...

# Conjunto de palavras-chave mais específicas para nanorevestimentos e tintas
palavras_chave_especificas = [
    # Termos principais
    "nanocoating", "nanocoatings", "nano coating", "nano coatings",
    "nanorevestimento", "nanorevestimentos",
    
    # Materiais nanoestruturados comuns
    "TiO2", "titanium dioxide", "zinc oxide", "ZnO", 
    "silica nanoparticle", "alumina nanoparticle",
    "carbon nanotube", "CNT", "graphene",
    
    # Propriedades funcionais
    "anticorrosive coating", "anti-corrosion coating",
    "antimicrobial coating", "antibacterial coating",
    "self-cleaning coating", "superhydrophobic coating",
    "photocatalytic coating",
    
    # Técnicas
    "sol-gel coating", "layer-by-layer", "dip coating",
    "spin coating", "spray coating",
    
    # Aplicações
    "protective coating", "functional coating",
    "smart coating", "intelligent coating"
]

def buscar_artigos_nanorevestimentos(df, palavras_chave, colunas=['Article Title', 'Abstract']):
    """
    Busca artigos usando múltiplas palavras-chave em múltiplas colunas
    """
    resultados_totais = pd.DataFrame()
    
    print("Buscando por palavras-chave específicas...")
    
    for palavra in palavras_chave:
        print(f"Buscando: '{palavra}'")
        
        for coluna in colunas:
            if coluna in df.columns:
                # Buscar na coluna
                temp_resultados = filtrar_coluna(df, coluna, palavra, tipo='contem')
                
                if len(temp_resultados) > 0:
                    print(f"  Encontrados {len(temp_resultados)} resultados em '{coluna}'")
                    resultados_totais = pd.concat([resultados_totais, temp_resultados])
    
    # Remover duplicatas
    if len(resultados_totais) > 0:
        resultados_totais = resultados_totais.drop_duplicates()
        print(f"\nTotal de artigos únicos encontrados: {len(resultados_totais)}")
    else:
        print("\nNenhum artigo encontrado com as palavras-chave especificadas.")
    
    return resultados_totais

# Carregar e examinar o DataFrame
df = pd.read_csv('/home/delon/Modelos/cenanoink/Projeto-CENanoInk/Alan Delon.csv')

print("Informações do DataFrame:")
print(f"Total de linhas: {len(df)}")
print(f"Colunas disponíveis: {list(df.columns)}")

# Buscar artigos relacionados a nanorevestimentos
artigos_encontrados = buscar_artigos_nanorevestimentos(df, palavras_chave_especificas)

if len(artigos_encontrados) > 0:
    print(f"\n=== ARTIGOS ENCONTRADOS ({len(artigos_encontrados)}) ===")
    
    # Mostrar primeiros resultados
    print("\nPrimeiros 10 títulos encontrados:")
    for i, titulo in enumerate(artigos_encontrados['Article Title'].head(10)):
        print(f"{i+1}. {titulo}")
    
    # Analisar com Gemini (limitado a 10 artigos para evitar problemas de API)
    print(f"\n=== ANÁLISE COM GEMINI (primeiros 10 artigos) ===")
    amostra_analise = artigos_encontrados.head(10)
    
    try:
        analise = analisar_com_gemini(amostra_analise)
        print("Análise do Gemini:")
        print(analise)
    except Exception as e:
        print(f"Erro na análise do Gemini: {e}")
    
    # Salvar resultados
    arquivo_saida = 'artigos_nanorevestimentos_encontrados.csv'
    artigos_encontrados.to_csv(arquivo_saida, index=False)
    print(f"\nArtigos salvos em: {arquivo_saida}")
    
    # Estatísticas por palavra-chave (opcional)
    print(f"\n=== ESTATÍSTICAS ===")
    print(f"Total de artigos encontrados: {len(artigos_encontrados)}")
    
    # Verificar se existe coluna Abstract para análise detalhada
    if 'Abstract' in artigos_encontrados.columns:
        abstracts_validos = artigos_encontrados['Abstract'].notna().sum()
        print(f"Artigos com abstract disponível: {abstracts_validos}")
        
        if abstracts_validos > 0:
            resposta = input("\nDeseja analisar os abstracts para adequação ao escopo? (s/n): ")
            if resposta.lower() == 's':
                print("Processando análise de escopo...")
                # Usar apenas uma amostra pequena para teste
                amostra_escopo = artigos_encontrados.head(5)
                resultado_escopo = processar_dataframe_escopo(amostra_escopo, batch_size=1)
                
                print("\nResultados da análise de escopo:")
                for idx, row in resultado_escopo.iterrows():
                    print(f"\nArtigo {idx + 1}:")
                    print(f"Título: {row.get('Article Title', 'N/A')[:80]}...")
                    print(f"Adequação: {row['Se adequa ao escopo?']}")

else:
    print("\nNenhum artigo encontrado. Vamos tentar com termos mais básicos:")
    
    # Termos mais básicos se não encontrar nada
    termos_basicos = ["coating", "paint", "nano", "nanoparticle", "surface"]
    
    for termo in termos_basicos:
        print(f"\nBuscando pelo termo básico: '{termo}'")
        resultado_basico = filtrar_coluna(df, 'Article Title', termo, tipo='contem')
        print(f"Encontrados {len(resultado_basico)} artigos com '{termo}' no título")
        
        if len(resultado_basico) > 0:
            print("Primeiros 5 títulos:")
            for titulo in resultado_basico['Article Title'].head(5):
                print(f"  - {titulo}")

Informações do DataFrame:
Total de linhas: 525
Colunas disponíveis: ['Publication Type', 'Authors', 'Book Authors', 'Book Editors', 'Book Group Authors', 'Author Full Names', 'Book Author Full Names', 'Group Authors', 'Article Title', 'Source Title', 'Book Series Title', 'Book Series Subtitle', 'Language', 'Document Type', 'Conference Title', 'Conference Date', 'Conference Location', 'Conference Sponsor', 'Conference Host', 'Author Keywords', 'Keywords Plus', 'Abstract', 'Addresses', 'Affiliations', 'Reprint Addresses', 'Email Addresses', 'Researcher Ids', 'ORCIDs', 'Funding Orgs', 'Funding Name Preferred', 'Funding Text', 'Cited References', 'Cited Reference Count', 'Times Cited, WoS Core', 'Times Cited, All Databases', '180 Day Usage Count', 'Since 2013 Usage Count', 'Publisher', 'Publisher City', 'Publisher Address', 'ISSN', 'eISSN', 'ISBN', 'Journal Abbreviation', 'Journal ISO Abbreviation', 'Publication Date', 'Publication Year', 'Volume', 'Issue', 'Part Number', 'Supplement', 'Sp

KeyboardInterrupt: 

## 4. Análise com Gemini AI

Funções para análise automática de abstracts usando a API do Gemini.

In [ ]:
# ...existing code...

def analisar_escopo_abstract(abstract_text):
    """
    Analisa um abstract individual para determinar se se adequa ao escopo de nanorevestimentos e tintas
    """
    if pd.isna(abstract_text) or str(abstract_text).strip() == '':
        return "Não se adequa - Abstract vazio ou não disponível"
    
    model = genai.GenerativeModel('gemini-2.0-flash')
    
    prompt = f"""
    Analise o seguinte abstract científico e determine se ele se adequa ao escopo de pesquisa sobre "nanorevestimentos e tintas (nanocoatings and paints)".

    Abstract: {abstract_text}

    Critérios para adequação ao escopo:
    - Estudos sobre nanomateriais aplicados em revestimentos ou tintas
    - Propriedades funcionais de nanocoatings (anticorrosão, antimicrobiano, autolimpante, etc.)
    - Técnicas de preparação ou caracterização de nanorevestimentos
    - Aplicações industriais de tintas nanoestruturadas
    - Materiais nanoestruturados para proteção de superfícies

    Responda APENAS uma das opções:
    1. "Sim - [breve justificativa]"
    2. "Não - [breve justificativa explicando por que não se adequa]"

    Mantenha a justificativa concisa (máximo 100 palavras).
    """
    
    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"Erro na análise - {str(e)}"

def processar_dataframe_escopo(df, coluna_abstract='Abstract', batch_size=10):
    """
    Processa todo o DataFrame adicionando a coluna de adequação ao escopo
    """
    # Verificar se a coluna Abstract existe
    if coluna_abstract not in df.columns:
        print(f"Coluna '{coluna_abstract}' não encontrada no DataFrame")
        return df
    
    # Criar uma cópia do DataFrame
    df_processado = df.copy()
    
    # Inicializar a nova coluna
    df_processado['Se adequa ao escopo?'] = ''
    
    total_linhas = len(df_processado)
    print(f"Processando {total_linhas} abstracts...")
    
    # Processar em lotes para evitar sobrecarga da API
    for i in range(0, total_linhas, batch_size):
        batch_end = min(i + batch_size, total_linhas)
        print(f"Processando linhas {i+1} a {batch_end}...")
        
        for idx in range(i, batch_end):
            abstract = df_processado.iloc[idx][coluna_abstract]
            resultado = analisar_escopo_abstract(abstract)
            df_processado.iloc[idx, df_processado.columns.get_loc('Se adequa ao escopo?')] = resultado
            
            # Mostrar progresso
            if (idx + 1) % 5 == 0:
                print(f"  Processado: {idx + 1}/{total_linhas}")
        
        # Pequena pausa entre lotes para não sobrecarregar a API
        import time
        time.sleep(2)
    
    return df_processado

# Carregar o DataFrame
df = pd.read_csv('/home/delon/Modelos/cenanoink/Projeto-CENanoInk/Alan Delon.csv')

print("Informações do DataFrame:")
print(f"Total de linhas: {len(df)}")
print(f"Colunas disponíveis: {list(df.columns)}")

# Verificar se existe coluna Abstract
if 'Abstract' in df.columns:
    print(f"Abstracts não vazios: {df['Abstract'].notna().sum()}")
    
    # Processar apenas uma amostra primeiro (para teste)
    print("\n=== PROCESSAMENTO DE TESTE (primeiras 5 linhas) ===")
    df_teste = df.head(5).copy()
    df_teste_processado = processar_dataframe_escopo(df_teste, batch_size=1)
    
    print("\nResultados do teste:")
    for idx, row in df_teste_processado.iterrows():
        print(f"\nLinha {idx + 1}:")
        print(f"Título: {row.get('Article Title', 'N/A')[:100]}...")
        print(f"Adequação: {row['Se adequa ao escopo?']}")
    
    # Perguntar se deseja processar todo o DataFrame
    resposta = input("\nDeseja processar todo o DataFrame? (s/n): ")
    if resposta.lower() == 's':
        print("\n=== PROCESSANDO TODO O DATAFRAME ===")
        df_completo = processar_dataframe_escopo(df)
        
        # Salvar resultado
        arquivo_saida = 'df_com_analise_escopo.csv'
        df_completo.to_csv(arquivo_saida, index=False)
        print(f"\nResultados salvos em: {arquivo_saida}")
        
        # Mostrar estatísticas
        adequados = df_completo['Se adequa ao escopo?'].str.contains('Sim', case=False, na=False).sum()
        nao_adequados = df_completo['Se adequa ao escopo?'].str.contains('Não', case=False, na=False).sum()
        
        print(f"\n=== ESTATÍSTICAS FINAIS ===")
        print(f"Adequados ao escopo: {adequados}")
        print(f"Não adequados ao escopo: {nao_adequados}")
        print(f"Total processado: {adequados + nao_adequados}")
        
else:
    print("Coluna 'Abstract' não encontrada no DataFrame")
    print("Colunas disponíveis:", list(df.columns))

Informações do DataFrame:
Total de linhas: 525
Colunas disponíveis: ['Publication Type', 'Authors', 'Book Authors', 'Book Editors', 'Book Group Authors', 'Author Full Names', 'Book Author Full Names', 'Group Authors', 'Article Title', 'Source Title', 'Book Series Title', 'Book Series Subtitle', 'Language', 'Document Type', 'Conference Title', 'Conference Date', 'Conference Location', 'Conference Sponsor', 'Conference Host', 'Author Keywords', 'Keywords Plus', 'Abstract', 'Addresses', 'Affiliations', 'Reprint Addresses', 'Email Addresses', 'Researcher Ids', 'ORCIDs', 'Funding Orgs', 'Funding Name Preferred', 'Funding Text', 'Cited References', 'Cited Reference Count', 'Times Cited, WoS Core', 'Times Cited, All Databases', '180 Day Usage Count', 'Since 2013 Usage Count', 'Publisher', 'Publisher City', 'Publisher Address', 'ISSN', 'eISSN', 'ISBN', 'Journal Abbreviation', 'Journal ISO Abbreviation', 'Publication Date', 'Publication Year', 'Volume', 'Issue', 'Part Number', 'Supplement', 'Sp

## 5. Processamento em Lote

Funções para processar grandes volumes de dados respeitando rate limits.

In [ ]:
import pandas as pd
import genaiimport random
as as pd
import time
import randomfrom typing import List

def analisar_escopo_abstract(abstract_text: str, retry_count: int = 3) -> str:lisar_escopo_abstract(abstract_text, retry_count=3):
    """
    Analisa um abstract individual para adequação ao escopoal para determinar se se adequa ao escopo de nanorevestimentos e tintas
    ndo Gemini 2.0 Flash-Lite
    Args:
        abstract_text: Texto do abstract:
        retry_count: Número de tentativas em caso de erro    return "Não se adequa - Abstract vazio ou não disponível"
    
    Returns:
        String com resultado da análisemodel = genai.GenerativeModel('gemini-2.0-flash-exp')
    """
    if pd.isna(abstract_text) or str(abstract_text).strip() == '':
        return "Não se adequa - Abstract vazio ou não disponível"    Analise o seguinte abstract científico e determine se ele se adequa ao escopo de pesquisa sobre "nanorevestimentos e tintas (nanocoatings and paints)".
    
    model = genai.GenerativeModel(CONFIG['modelo_gemini'])    Abstract: {abstract_text}
    
    prompt = f"""
    Analise o seguinte abstract científico e determine se ele se adequa ao escopo de pesquisa sobre "nanorevestimentos e tintas (nanocoatings and paints)".
crobiano, autolimpante, etc.)
    Abstract: {abstract_text}evestimentos

    Critérios para adequação ao escopo:    - Materiais nanoestruturados para proteção de superfícies
    - Estudos sobre nanomateriais aplicados em revestimentos ou tintas
    - Propriedades funcionais de nanocoatings (anticorrosão, antimicrobiano, autolimpante, etc.)
    - Técnicas de preparação ou caracterização de nanorevestimentos
    - Aplicações industriais de tintas nanoestruturadas    2. "Não - [breve justificativa explicando por que não se adequa]"
    - Materiais nanoestruturados para proteção de superfícies
tenha a justificativa concisa (máximo 100 palavras).
    Responda APENAS uma das opções:"""
    1. "Sim - [breve justificativa]"
    2. "Não - [breve justificativa explicando por que não se adequa]"mpt in range(retry_count):

    Mantenha a justificativa concisa (máximo 100 palavras).ntent(prompt)
    """xt.strip()
    
    for attempt in range(retry_count):
        try:in error_msg.lower():
            response = model.generate_content(prompt)
            return response.text.strip()ar rate limit
        except Exception as e:
            error_msg = str(e)- aguardando {wait_time}s...")
            if "429" in error_msg or "quota" in error_msg.lower():ep(wait_time)
                if attempt < retry_count - 1:ontinue
                    wait_time = 3 + random.randint(1, 2)
                    print(f"  ⏳ Rate limit - aguardando {wait_time}s...")   return f"Erro - Rate limit excedido após {retry_count} tentativas"
                    time.sleep(wait_time)
                    continue            return f"Erro na análise - {error_msg}"
                else:
                    return f"Erro - Rate limit excedido após {retry_count} tentativas"    return "Erro - Falha após múltiplas tentativas"
            else:
                              coluna_abstract: str = 'Abstract') -> pd.DataFrame:         return f"Erro na análise - {error_msg}"cessar_dataframe_escopo(df, coluna_abstract='Abstract', max_requests_per_day=1400):
    """
    Processa DataFrame completo adicionando análise de escopo" de adequação ao escopo
    Respeitando limites de API: 30 req/min, 1.500 req/diao limites: 30 req/min, 1.500 req/dia
    bstract='Abstract', max_requests_per_day=1400):
    Args:
        df: DataFrame a ser processadoo
        coluna_abstract: Nome da coluna com abstractsimites: 30 req/min, 1.500 req/diaoluna '{coluna_abstract}' não encontrada no DataFrame")
    """    return df
    Returns:ct existe
        DataFrame com nova coluna 'Se adequa ao escopo?' df.columns:Frame
    """    print(f"Coluna '{coluna_abstract}' não encontrada no DataFrame")df_processado = df.copy()
    if coluna_abstract not in df.columns:
        print(f"❌ Coluna '{coluna_abstract}' não encontrada")
        return df# Criar uma cópia do DataFramedf_processado['Se adequa ao escopo?'] = ''
    
    df_processado = df.copy()
    df_processado['Se adequa ao escopo?'] = ''# Inicializar a nova colunaprint(f"Processando {total_linhas} abstracts...")
    '
    total_linhas = len(df_processado)
    print(f"📋 Processando {total_linhas} abstracts...")
    
    # Verificar limite diárioax_requests_per_day} linhas? (s/n): ")
    if total_linhas > CONFIG['max_requests_per_day']:
        print(f"⚠️ Dataset tem {total_linhas} linhas, limite diário: {CONFIG['max_requests_per_day']}")linhas > max_requests_per_day:otal_linhas = max_requests_per_day
        resposta = input(f"Processar apenas as primeiras {CONFIG['max_requests_per_day']} linhas? (s/n): ")inhas} linhas, mas limite diário é {max_requests_per_day}")
        if resposta.lower() == 's':a processar apenas as primeiras {max_requests_per_day} linhas? (s/n): ") cancelado.")
            total_linhas = CONFIG['max_requests_per_day']    if resposta.lower() == 's':        return df_processado
        else:
            print("❌ Processamento cancelado")
            return df_processado        print("Processamento cancelado.")print(f"Tempo estimado: {(total_linhas * 2.5) / 60:.1f} minutos")
    _processado
    tempo_estimado = (total_linhas * CONFIG['delay_between_requests']) / 60
    print(f"⏱️ Tempo estimado: {tempo_estimado:.1f} minutos")print(f"Rate limiting: 30 req/min, processando {total_linhas} abstracts")start_time = time.time()
    print(f"🔄 Rate limiting: {CONFIG['max_requests_per_minute']} req/min")_linhas * 2.5) / 60:.1f} minutos")
    
    requests_made = 0
    start_time = time.time()
    adequados_count = 0
    
    for idx in range(total_linhas): 30 req/min aguardando {wait_time:.1f}s...")
        # Controle de rate limitime() - start_timeime)
        elapsed_time = time.time() - start_timelapsed_time < 60:
        if requests_made >= (CONFIG['max_requests_per_minute'] - 1) and elapsed_time < 60:    wait_time = 60 - elapsed_time + 2  # +2 segundos de margem    start_time = time.time()
            wait_time = 60 - elapsed_time + 2}s...")
            print(f"\n⏳ Rate limit: aguardando {wait_time:.1f}s...")    time.sleep(wait_time)abstract = df_processado.iloc[idx][coluna_abstract]
            time.sleep(wait_time) 0
            requests_made = 0
            start_time = time.time()
        abstract = df_processado.iloc[idx][coluna_abstract]    print(f"Processando {idx + 1}/{total_linhas} ({((idx + 1)/total_linhas)*100:.1f}%)")
        abstract = df_processado.iloc[idx][coluna_abstract]
        
        # Progressoif idx % 10 == 0 or idx == total_linhas - 1:df_processado.iloc[idx, df_processado.columns.get_loc('Se adequa ao escopo?')] = resultado
        if idx % 10 == 0 or idx == total_linhas - 1:sando {idx + 1}/{total_linhas} ({((idx + 1)/total_linhas)*100:.1f}%)")
            print(f"📊 Processando {idx + 1}/{total_linhas} ({((idx + 1)/total_linhas)*100:.1f}%)")requests_made += 1
        
        resultado = analisar_escopo_abstract(abstract)entre requisições (2 segundos para 30 req/min)loc[idx, df_processado.columns.get_loc('Se adequa ao escopo?')] = resultado
        df_processado.iloc[idx, df_processado.columns.get_loc('Se adequa ao escopo?')] = resultadotime.sleep(2.1)
        
        if 'Sim' in resultado:ada 25 processamentos
            adequados_count += 1
        +1].str.contains('Sim', case=False, na=False).sum()
        requests_made += 1        print(f"  Adequados até agora: {adequados_parcial}/{idx + 1}")    
        time.sleep(CONFIG['delay_between_requests'])samentos
            return df_processado        if (idx + 1) % 25 == 0:
        # Estatísticas parciais
        if (idx + 1) % 25 == 0:     print(f"  Adequados até agora: {adequados_parcial}/{idx + 1}")cessar_com_checkpoint(df, coluna_abstract='Abstract', checkpoint_file='checkpoint_analise.csv', start_from=0):
            taxa_atual = (adequados_count / (idx + 1)) * 100
            print(f"  ✅ Adequados até agora: {adequados_count}/{idx + 1} ({taxa_atual:.1f}%)")urn df_processadosão com checkpoint para poder retomar o processamento
    
    return df_processadotract='Abstract', checkpoint_file='checkpoint_analise.csv', start_from=0):
_file and start_from == 0:
def processar_com_checkpoint(df: pd.DataFrame, ento
                           coluna_abstract: str = 'Abstract') -> pd.DataFrame:
    """
    Processamento com sistema de checkpoint para recuperação
    )
    Args:checkpoint_file)s':
        df: DataFrame a ser processadopo?' in df_checkpoint.columns:ckpoint
        coluna_abstract: Nome da coluna com abstractslinhas já processadas")
                resposta = input("Deseja continuar do checkpoint? (s/n): ")        print("Nenhum checkpoint encontrado, iniciando do zero")
    Returns:.lower() == 's':
        DataFrame processado
    """    except FileNotFoundError:df_processado = processar_dataframe_escopo(df, coluna_abstract)
    checkpoint_file = CONFIG['checkpoint_file']um checkpoint encontrado, iniciando do zero")
    
    # Verificar checkpoint existente
    try:a_abstract)e)
        df_checkpoint = pd.read_csv(checkpoint_file)    print(f"Checkpoint salvo em: {checkpoint_file}")
        if 'Se adequa ao escopo?' in df_checkpoint.columns:
            processed_count = df_checkpoint['Se adequa ao escopo?'].ne('').sum()    if checkpoint_file:    return df_processado
            print(f"📂 Checkpoint encontrado: {processed_count} linhas já processadas")to_csv(checkpoint_file, index=False)
            
            resposta = input("Continuar do checkpoint? (s/n): ")        """
            if resposta.lower() == 's': específica
                return df_checkpoint
    except FileNotFoundError:.DataFrame) -> str:
        print("📝 Nenhum checkpoint encontrado, iniciando do zero")    """        df: DataFrame a ser filtrado
    rtigos com Gemini
    # Processar normalmente
    df_processado = processar_dataframe_escopo(df, coluna_abstract)
    s
    # Salvar checkpointReturns:
    df_processado.to_csv(checkpoint_file, index=False)
    print(f"💾 Checkpoint salvo em: {checkpoint_file}")
    
    return df_processadoif len(dados_filtrados) == 0:    print(f"⚠️ Coluna '{coluna}' não encontrada")
aFrame()m dado disponível para análise."
# Carregar o DataFrame
df = pd.read_csv('/home/delon/Modelos/cenanoink/Projeto-CENanoInk/Alan Delon.csv')
str.contains(termo, case=False, na=False)filtrados.head(10)
print("Informações do DataFrame:")elif tipo == 'exato':texto_dados = dados_amostra[['Article Title', 'Abstract']].to_string()
print(f"Total de linhas: {len(df)}")
print(f"Colunas disponíveis: {list(df.columns)}")delo_gemini'])
h(termo.lower())
# Verificar se existe coluna Abstract
if 'Abstract' in df.columns:s e tintas:
    abstracts_validos = df['Abstract'].notna().sum()
    print(f"Abstracts não vazios: {abstracts_validos}")return df[mask]{texto_dados}
    
    # Teste com 3 abstracts primeiroataFrame, 
    print("\n=== TESTE INICIAL (3 abstracts) ===")
    df_teste = df.head(3).copy()['Article Title', 'Abstract']) -> pd.DataFrame:
    
    # Processar testeBusca artigos usando múltiplas palavras-chave em múltiplas colunas4. Aplicações industriais identificadas
    start_test = time.time()
    df_teste_processado = processar_dataframe_escopo(df_teste)
    end_test = time.time()
    
    print(f"\nTempo do teste: {end_test - start_test:.1f} segundos")colunas: Colunas onde buscar
    print("\nResultados do teste:")
    for idx, row in df_teste_processado.iterrows():
        print(f"\nLinha {idx + 1}:")
        print(f"Título: {row.get('Article Title', 'N/A')[:80]}...")
        print(f"Adequação: {row['Se adequa ao escopo?']}")ltados = pd.DataFrame()return f"Erro na análise geral: {str(e)}"
    
    # Perguntar sobre processamento completo
    print(f"\n=== PROCESSAMENTO COMPLETO ===")
    print(f"Total de abstracts para processar: {abstracts_validos}")
    print(f"Limite diário recomendado: 1.400 requisições")palavra in palavras_chave:nformações do DataFrame:")
    print(f"Tempo estimado: {(abstracts_validos * 2.5) / 60:.1f} minutos")
    
    resposta = input("\nDeseja processar todo o DataFrame? (s/n): ")
    if resposta.lower() == 's':oluna(df, coluna, palavra, tipo='contem')tract
        print("\nIniciando processamento completo com checkpoint...")
        df_completo = processar_com_checkpoint(df, checkpoint_file='analise_escopo_checkpoint.csv')            count_palavra += len(temp)racts_validos = df['Abstract'].notna().sum()
        tados = pd.concat([resultados, temp]) vazios: {abstracts_validos}")
        # Salvar resultado final
        arquivo_saida = 'df_com_analise_escopo_completo.csv'if count_palavra > 0:ste com 3 abstracts primeiro
        df_completo.to_csv(arquivo_saida, index=False)       estatisticas[palavra] = count_palavrarint("\n=== TESTE INICIAL (3 abstracts) ===")
        print(f"\nResultados finais salvos em: {arquivo_saida}")dos")
        

















    print("Colunas disponíveis:", list(df.columns))    print("Coluna 'Abstract' não encontrada no DataFrame")else:                    print(f"Taxa de adequação: {(adequados/(adequados + nao_adequados))*100:.1f}%")        if adequados > 0:                print(f"Total processado: {adequados + nao_adequados + erros}")        print(f"Erros: {erros}")        print(f"Não adequados ao escopo: {nao_adequados}")        print(f"Adequados ao escopo: {adequados}")        print(f"\n=== ESTATÍSTICAS FINAIS ===")                erros = df_completo['Se adequa ao escopo?'].str.contains('Erro', case=False, na=False).sum()        nao_adequados = df_completo['Se adequa ao escopo?'].str.contains('Não', case=False, na=False).sum()        adequados = df_completo['Se adequa ao escopo?'].str.contains('Sim', case=False, na=False).sum()        # Mostrar estatísticas






































































    print("Colunas disponíveis:", list(df.columns))    print("Coluna 'Abstract' não encontrada no DataFrame")else:                    print(f"Taxa de adequação: {(adequados/(adequados + nao_adequados))*100:.1f}%")        if adequados > 0:                print(f"Total processado: {adequados + nao_adequados + erros}")        print(f"Erros: {erros}")        print(f"Não adequados ao escopo: {nao_adequados}")        print(f"Adequados ao escopo: {adequados}")        print(f"\n=== ESTATÍSTICAS FINAIS ===")                erros = df_completo['Se adequa ao escopo?'].str.contains('Erro', case=False, na=False).sum()        nao_adequados = df_completo['Se adequa ao escopo?'].str.contains('Não', case=False, na=False).sum()        adequados = df_completo['Se adequa ao escopo?'].str.contains('Sim', case=False, na=False).sum()        # Mostrar estatísticas                print(f"\nResultados finais salvos em: {arquivo_saida}")        df_completo.to_csv(arquivo_saida, index=False)        arquivo_saida = 'df_com_analise_escopo_completo.csv'        # Salvar resultado final                df_completo = processar_com_checkpoint(df, checkpoint_file='analise_escopo_checkpoint.csv')        print("\nIniciando processamento completo com checkpoint...")    if resposta.lower() == 's':    resposta = input("\nDeseja processar todo o DataFrame? (s/n): ")        print(f"Tempo estimado: {(abstracts_validos * 2.5) / 60:.1f} minutos")    print(f"Limite diário recomendado: 1.400 requisições")    print(f"Total de abstracts para processar: {abstracts_validos}")    print(f"\n=== PROCESSAMENTO COMPLETO ===")    # Perguntar sobre processamento completo            print(f"Adequação: {row['Se adequa ao escopo?']}")        print(f"Título: {row.get('Article Title', 'N/A')[:80]}...")        print(f"\nLinha {idx + 1}:")    for idx, row in df_teste_processado.iterrows():    print("\nResultados do teste:")    print(f"\nTempo do teste: {end_test - start_test:.1f} segundos")        end_test = time.time()    df_teste_processado = processar_dataframe_escopo(df_teste)    start_test = time.time()    # Processar teste        df_teste = df.head(3).copy()    print("\n=== TESTE INICIAL (3 abstracts) ===")    # Teste com 3 abstracts primeiro        print(f"Abstracts não vazios: {abstracts_validos}")    abstracts_validos = df['Abstract'].notna().sum()if 'Abstract' in df.columns:# Verificar se existe coluna Abstractprint(f"Colunas disponíveis: {list(df.columns)}")print(f"Total de linhas: {len(df)}")print("Informações do DataFrame:")df = pd.read_csv('/home/delon/Modelos/cenanoink/Projeto-CENanoInk/Alan Delon.csv')# Carregar o DataFrame    return resultados            print("\n❌ Nenhum artigo encontrado")    else:        print(f"📊 Palavras-chave com resultados: {len(estatisticas)}")        print(f"\n✅ Total de artigos únicos encontrados: {len(resultados)}")        resultados = resultados.drop_duplicates()    if len(resultados) > 0:    # Remover duplicatas    # Processar teste
    start_test = time.time()
    df_teste_processado = processar_dataframe_escopo(df_teste)
    end_test = time.time()
    
    print(f"\nTempo do teste: {end_test - start_test:.1f} segundos")
    print("\nResultados do teste:")
    for idx, row in df_teste_processado.iterrows():
        print(f"\nLinha {idx + 1}:")
        print(f"Título: {row.get('Article Title', 'N/A')[:80]}...")
        print(f"Adequação: {row['Se adequa ao escopo?']}")
    
    # Perguntar sobre processamento completo
    print(f"\n=== PROCESSAMENTO COMPLETO ===")
    print(f"Total de abstracts para processar: {abstracts_validos}")
    print(f"Limite diário recomendado: 1.400 requisições")
    print(f"Tempo estimado: {(abstracts_validos * 2.5) / 60:.1f} minutos")
    
    resposta = input("\nDeseja processar todo o DataFrame? (s/n): ")
    if resposta.lower() == 's':
        print("\nIniciando processamento completo com checkpoint...")
        df_completo = processar_com_checkpoint(df, checkpoint_file='analise_escopo_checkpoint.csv')
        
        # Salvar resultado final
        arquivo_saida = 'df_com_analise_escopo_completo.csv'
        df_completo.to_csv(arquivo_saida, index=False)
        print(f"\nResultados finais salvos em: {arquivo_saida}")
        
        # Mostrar estatísticas
        adequados = df_completo['Se adequa ao escopo?'].str.contains('Sim', case=False, na=False).sum()
        nao_adequados = df_completo['Se adequa ao escopo?'].str.contains('Não', case=False, na=False).sum()
        erros = df_completo['Se adequa ao escopo?'].str.contains('Erro', case=False, na=False).sum()
        
        print(f"\n=== ESTATÍSTICAS FINAIS ===")
        print(f"Adequados ao escopo: {adequados}")
        print(f"Não adequados ao escopo: {nao_adequados}")
        print(f"Erros: {erros}")
        print(f"Total processado: {adequados + nao_adequados + erros}")
        
        if adequados > 0:
            print(f"Taxa de adequação: {(adequados/(adequados + nao_adequados))*100:.1f}%")
        
else:
    print("Coluna 'Abstract' não encontrada no DataFrame")
    print("Colunas disponíveis:", list(df.columns))

## 6. Execução Principal

Script principal para carregar dados e executar análises.

In [ ]:
# Carregar e explorar os dados
print("📂 Carregando dados...")

try:
    df = pd.read_csv(CONFIG['arquivo_entrada'])
    print(f"✅ Dados carregados com sucesso")
    print(f"📊 Total de linhas: {len(df):,}")
    print(f"📊 Total de colunas: {len(df.columns)}")
    print(f"📋 Colunas disponíveis: {list(df.columns)}")
    
    # Verificar colunas essenciais
    colunas_essenciais = ['Article Title', 'Abstract']
    colunas_encontradas = [col for col in colunas_essenciais if col in df.columns]
    colunas_faltantes = [col for col in colunas_essenciais if col not in df.columns]
    
    print(f"\n✅ Colunas encontradas: {colunas_encontradas}")
    if colunas_faltantes:
        print(f"⚠️ Colunas faltantes: {colunas_faltantes}")
    
    # Estatísticas dos abstracts
    if 'Abstract' in df.columns:
        abstracts_validos = df['Abstract'].notna().sum()
        abstracts_vazios = df['Abstract'].isna().sum()
        print(f"\n📄 Abstracts disponíveis: {abstracts_validos:,}")
        print(f"📄 Abstracts vazios: {abstracts_vazios:,}")
        print(f"📄 Taxa de completude: {(abstracts_validos/len(df))*100:.1f}%")
    
except FileNotFoundError:
    print(f"❌ Arquivo não encontrado: {CONFIG['arquivo_entrada']}")
    df = None
except Exception as e:
    print(f"❌ Erro ao carregar dados: {e}")
    df = None

In [ ]:
# Buscar artigos relevantes por palavras-chave
if df is not None:
    print("\n" + "="*50)
    print("🔍 BUSCA POR PALAVRAS-CHAVE")
    print("="*50)
    
    # Buscar artigos usando todas as palavras-chave
    artigos_encontrados = buscar_artigos_por_palavras_chave(df, todas_palavras_chave)
    
    if len(artigos_encontrados) > 0:
        print(f"\n📋 RESULTADOS DA BUSCA:")
        print(f"  Total de artigos encontrados: {len(artigos_encontrados):,}")
        print(f"  Percentual do dataset: {(len(artigos_encontrados)/len(df))*100:.2f}%")
        
        # Mostrar amostra dos títulos
        print(f"\n📑 Primeiros 10 títulos encontrados:")
        for i, titulo in enumerate(artigos_encontrados['Article Title'].head(10), 1):
            print(f"  {i:2d}. {titulo[:80]}{'...' if len(titulo) > 80 else ''}")
        
        # Salvar artigos encontrados
        arquivo_filtrados = 'artigos_nanorevestimentos_encontrados.csv'
        artigos_encontrados.to_csv(arquivo_filtrados, index=False)
        print(f"\n💾 Artigos filtrados salvos em: {arquivo_filtrados}")
        
        # Análise geral com Gemini
        print(f"\n🤖 Realizando análise geral com Gemini...")
        analise_geral = analisar_com_gemini_geral(artigos_encontrados)
        print("\n" + "="*50)
        print("📊 ANÁLISE GERAL - GEMINI AI")
        print("="*50)
        print(analise_geral)
        
    else:
        print("\n❌ Nenhum artigo encontrado com as palavras-chave especificadas")
        print("💡 Sugestão: Verificar se o dataset contém artigos da área")
else:
    print("❌ Não foi possível carregar os dados")

In [ ]:
# Análise detalhada de escopo (opcional)
if df is not None and len(artigos_encontrados) > 0:
    print("\n" + "="*50)
    print("🔬 ANÁLISE DETALHADA DE ESCOPO")
    print("="*50)
    
    if 'Abstract' in artigos_encontrados.columns:
        abstracts_disponiveis = artigos_encontrados['Abstract'].notna().sum()
        print(f"📄 Artigos com abstract: {abstracts_disponiveis:,}")
        
        if abstracts_disponiveis > 0:
            print(f"⚠️ ATENÇÃO: Esta análise usa a API do Gemini")
            print(f"   Limite: {CONFIG['max_requests_per_day']} requisições/dia")
            print(f"   Tempo estimado: {(abstracts_disponiveis * CONFIG['delay_between_requests'])/60:.1f} minutos")
            
            resposta = input("\nRealizar análise detalhada de todos os abstracts? (s/n): ")
            
            if resposta.lower() == 's':
                print("\n🚀 Iniciando análise detalhada com checkpoint...")
                
                # Usar apenas artigos encontrados para análise
                df_analisado = processar_com_checkpoint(artigos_encontrados)
                
                # Salvar resultado final
                df_analisado.to_csv(CONFIG['arquivo_saida'], index=False)
                print(f"\n💾 Análise completa salva em: {CONFIG['arquivo_saida']}")
                
                # Estatísticas finais
                print("\n" + "="*50)
                print("📊 ESTATÍSTICAS FINAIS")
                print("="*50)
                
                adequados = df_analisado['Se adequa ao escopo?'].str.contains('Sim', case=False, na=False).sum()
                nao_adequados = df_analisado['Se adequa ao escopo?'].str.contains('Não', case=False, na=False).sum()
                erros = df_analisado['Se adequa ao escopo?'].str.contains('Erro', case=False, na=False).sum()
                total_analisado = adequados + nao_adequados + erros
                
                print(f"✅ Adequados ao escopo: {adequados:,}")
                print(f"❌ Não adequados: {nao_adequados:,}")
                print(f"⚠️ Erros: {erros:,}")
                print(f"📊 Total analisado: {total_analisado:,}")
                
                if adequados + nao_adequados > 0:
                    taxa_adequacao = (adequados / (adequados + nao_adequados)) * 100
                    print(f"📈 Taxa de adequação: {taxa_adequacao:.1f}%")
                
                # Mostrar alguns exemplos
                if adequados > 0:
                    print(f"\n📑 Exemplos de artigos adequados:")
                    adequados_df = df_analisado[df_analisado['Se adequa ao escopo?'].str.contains('Sim', case=False, na=False)]
                    for i, row in adequados_df.head(3).iterrows():
                        print(f"\n{i+1}. {row['Article Title'][:80]}...")
                        print(f"   Adequação: {row['Se adequa ao escopo?']}")
            
            else:
                print("\n⏭️ Análise detalhada pulada")
                
                # Fazer teste com 3 abstracts
                print("\n🧪 Realizando teste com 3 abstracts...")
                amostra_teste = artigos_encontrados.head(3)
                df_teste = processar_dataframe_escopo(amostra_teste)
                
                print("\n📋 Resultados do teste:")
                for idx, row in df_teste.iterrows():
                    print(f"\n{idx+1}. {row['Article Title'][:60]}...")
                    print(f"   Adequação: {row['Se adequa ao escopo?']}")
        else:
            print("❌ Nenhum abstract disponível para análise")
    else:
        print("❌ Coluna 'Abstract' não encontrada")

## 7. Conclusão

### Funcionalidades implementadas:

1. ✅ **Busca por palavras-chave**: Sistema robusto de filtro com múltiplas categorias
2. ✅ **Análise com Gemini AI**: Análise automática de relevância dos abstracts
3. ✅ **Rate limiting**: Respeita limites da API (30 req/min, 1.500 req/dia)
4. ✅ **Sistema de checkpoint**: Permite retomar análises interrompidas
5. ✅ **Estatísticas detalhadas**: Métricas de adequação e performance
6. ✅ **Exportação de dados**: Salva resultados em formato CSV

### Arquivos gerados:
- `artigos_nanorevestimentos_encontrados.csv`: Artigos filtrados por palavras-chave
- `analise_escopo_checkpoint.csv`: Checkpoint do processamento
- `df_com_analise_escopo_completo.csv`: Análise completa com Gemini

### Próximos passos:
- Refinar palavras-chave com base nos resultados
- Implementar análise de trends temporais
- Adicionar análise de colaborações e redes
- Desenvolver dashboard interativo